# Chatbot Implementation: Drug Information Assistant

This notebook shows the implementation of our multilingual medical (drug related) chatbot, including:

- PDF parsing and sectioning
- Vector store (database) creation using FAISS for semantic search
- Question-answering pipeline with LLM integration
- Language detection and automatic translation


## Setup & Imports


In [ ]:
!pip install langchain langchain-community faiss-cpu sentence-transformers transformers sentencepiece requests tqdm pypdf torch langdetect langcodes pycountry
!pip install transformers bert-score

In [66]:
import os
import re
import time
import requests
import torch
import difflib
import pycountry
import pandas as pd

from collections import Counter
from difflib import SequenceMatcher
from tqdm import tqdm
from langdetect import detect, detect_langs
from langcodes import Language
from transformers import (AutoTokenizer,AutoModelForSeq2SeqLM,MarianMTModel,MarianTokenizer,M2M100ForConditionalGeneration,M2M100Tokenizer)
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from bert_score import score as bert_score

## Structured Extraction of Drug Information PDFs Sections

This section focuses on parsing medical drug leaflets (PDF format) and segmenting them into semantically meaningful sections based on their internal structure. Specifically, we extract the sections listed under the heading: “What is in this leaflet”

In [53]:
def create_tolerant_spaces(title):
    
    # Converts a section title into a whitespace-tolerant regex pattern
    words = re.split(r"\s+", title.strip())
    pattern = r"\s+".join(map(re.escape, words))
    return pattern

def extract_sections_from_pdf(pdf_path):
    
    # Load the full content of the PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    full_text = "\n".join([p.page_content for p in pages])
    lines = full_text.splitlines()

    # Locate the index section titled "What is in this leaflet"
    leaflet_index = next((i for i, line in enumerate(lines) if "what is in this leaflet" in line.lower()), -1)

    if leaflet_index == -1 or leaflet_index + 6 >= len(lines):
        return []  # Index not found or not enough lines following it (there must be 6 sections)

    # Extract the next 6 lines as section titles
    section_titles = [lines[leaflet_index + i].strip(" .:\u2022") for i in range(1, 7)]
    section_titles = [t for t in section_titles if t]

    if len(section_titles) < 6:
        print(f"Could not find 6 section titles in {os.path.basename(pdf_path)}")
        return []

    # Extract the drug name from the file name
    med_name = os.path.splitext(os.path.basename(pdf_path))[0].replace("_", " ")

    # Skip the index part to start searching in the main content
    text = "\n".join(lines[leaflet_index + 7:])

    documents = []
    found_sections = 0

    # Iterate over each expected section title
    for i, title in enumerate(section_titles):
        current = create_tolerant_spaces(title)
        next_title = create_tolerant_spaces(section_titles[i + 1]) if i < len(section_titles) - 1 else None

        # Build pattern to extract text between this title and the next (or until the end if it's the last)
        if next_title:
            pattern = rf"{current}(.+?){next_title}"
        else:
            pattern = rf"{current}(.+)"

        # Search for the section content using the pattern
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)

        if match:
            
            body = match.group(1).strip()
            # Only keep the section if it's not too short
            
            if len(body) >= 100:
                
                # Construct a full chunk including the med name, title, and body
                full_chunk = f"{med_name}\n{title}\n{body}"
                
                # Store it as a LangChain Document object with source metadata
                documents.append(Document(
                    page_content=full_chunk,
                    metadata={"source": os.path.basename(pdf_path)}
                ))
                found_sections += 1

    # Return the list of structured Document chunks
    return documents


Example of PDF Section Extraction : this code extracts and prints the main sections from a drug leaflet using the extract_sections_from_pdf function.


In [54]:
# Set the name of the PDF to be processed 
pdf_name = "ACULAR_0_5_w_v_EYE_DROPS_SOLUTION_Ketorolac_trometamol_.pdf"
pdf_path = os.path.join("named_database", pdf_name)

# Process the PDF and extract sections
print(f"Processing: {pdf_name}\n")
secciones = extract_sections_from_pdf(pdf_path)

# Check if any sections were extracted and show them

if not secciones:
    print("Warning: No sections with useful content were found.")
else:
    for i, doc in enumerate(secciones, 1):
        print(f"\nSection {i} (Source: {doc.metadata['source']}):\n")
        print(doc.page_content)
        print("\n" + "-"*80)


Processing: ACULAR_0_5_w_v_EYE_DROPS_SOLUTION_Ketorolac_trometamol_.pdf


Section 1 (Source: ACULAR_0_5_w_v_EYE_DROPS_SOLUTION_Ketorolac_trometamol_.pdf):

ACULAR 0 5 w v EYE DROPS SOLUTION Ketorolac trometamol 
1. What ACULAR is and what it is used for
Acular is used to prevent and reduce eye inflammation 
following cataract surgery in adults. Acular belongs to a 
group of medicines known as non-steroidal anti-
inflammatory drugs (NSAIDS).

--------------------------------------------------------------------------------

Section 2 (Source: ACULAR_0_5_w_v_EYE_DROPS_SOLUTION_Ketorolac_trometamol_.pdf):

ACULAR 0 5 w v EYE DROPS SOLUTION Ketorolac trometamol 
2. What you need to know before you use ACULAR
Do not use ACULAR  
 If you are allergic to ketorolac trometamol or any of 
the other ingredients of this medicine (listed in section 
6).  
 If you are allergic to aspirin or any other similar drugs, 
such as other non-steroidal anti-inflammatory drugs.  
Warnings and precautions  
T

Processing all PDFs in the Dataset: this section iterates over all PDF files in the dataset folder, extracts the leaflet sections from each one, and tracks how many were successfully parsed.


In [55]:
all_docs = []
pdf_dir = "named_database"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

print(f"Total PDFs found: {len(pdf_files)}\n")

# Counters
six_sections_count = 0
few_sections_files = []

for i, pdf_path in enumerate(pdf_files, 1):
    try:
        docs = extract_sections_from_pdf(pdf_path)
        all_docs.extend(docs)

        # Display number of extracted sections per file
        print(f"[{i}/{len(pdf_files)}] {os.path.basename(pdf_path)} ➜ {len(docs)} useful sections")

        if len(docs) == 6:
            six_sections_count += 1
        else:
            few_sections_files.append(os.path.basename(pdf_path))

    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

# Final summary
print(f"\nTotal sections processed: {len(all_docs)}")
print(f"Documents with exactly 6 sections: {six_sections_count}")
print(f"Documents with a different number of sections: {len(few_sections_files)}")


Total PDFs found: 2798

[1/2798] 4-Dichlorobenzyl_alcohol_Mylan_0_6mg.pdf ➜ 6 useful sections
[2/2798] Abacavir_Lamivudine_Mylan_600_mg_300_mg_film-coated_tablets.pdf ➜ 6 useful sections
[3/2798] Abacavir_Lamivudine_Mylan_Pharma_600_mg_300_mg_Film-coated_Tablets.pdf ➜ 6 useful sections
[4/2798] Abiraterone_500_mg_film-coated_tablets.pdf ➜ 6 useful sections
[5/2798] Abiraterone_Aristo_250_mg_tablets.pdf ➜ 6 useful sections
[6/2798] Abiraterone_Aristo_500_mg_film-coated_tablets.pdf ➜ 6 useful sections
[7/2798] Abiraterone_Clonmel_500_mg_film-coated_tablets_abiraterone_acetate.pdf ➜ 6 useful sections
[8/2798] Abiraterone_Rowa_500_mg_film-coated_tablets_abiraterone_acetate.pdf ➜ 6 useful sections
[9/2798] Abiraterone_Teva_500_mg_film-coated_tablets_abiraterone_acetate.pdf ➜ 6 useful sections
[10/2798] Abstral_100_microgram_sublingual_tablets.pdf ➜ 6 useful sections
[11/2798] Abstral_100_microgram_sublingual_tablets_Abstral_200_microgram_sublingual_tablets.pdf ➜ 6 useful sections
[12/2798] 

[93/2798] Amitriptyline_hydrochloride_50mg_5ml_Oral_Solution.pdf ➜ 6 useful sections
[94/2798] Amlode_5mg.pdf ➜ 6 useful sections
[95/2798] Amlode_5_mg_tablets.pdf ➜ 6 useful sections
[96/2798] Amlodipine_20mg.pdf ➜ 6 useful sections
[97/2798] Amlodipine_5mg.pdf ➜ 6 useful sections
[98/2798] Amlodipine_5_mg_tablets_Amlodipine_10_mg_tablets.pdf ➜ 6 useful sections
[99/2798] Amlodipine_Bluefish_5_mg_tablets.pdf ➜ 6 useful sections
[100/2798] Amlodipine_Bluefish_5_mg_tablets_Amlodipine_Bluefish_10_mg_tablets.pdf ➜ 6 useful sections
[101/2798] Amlodipine_Clonmel_5mg_Tablets.pdf ➜ 6 useful sections
[102/2798] Amlodipine_Clonmel_5mg_Tablets_Amlodipine_Clonmel_10mg_Tablets.pdf ➜ 6 useful sections
[103/2798] Amlodipine_Fair-Med_5mg.pdf ➜ 6 useful sections
[104/2798] Amlodipine_Fair-Med_5mg_Tablets_Amlodipine_Fair-Med_10mg_Tablets.pdf ➜ 6 useful sections
[105/2798] amlodipine_Krka_4mg.pdf ➜ 6 useful sections
[106/2798] Amlodipine_Krka_5_mg_tablets.pdf ➜ 6 useful sections
[107/2798] Amlodipine_T

[194/2798] Arovi_4_000_IU_40_mg_0_4_mL_solution_for_injection_in_pre-filled_syringe.pdf ➜ 6 useful sections
[195/2798] Arovi_6_000_IU_60_mg_0_6_mL_solution_for_injection_in_pre-filled_syringe_enoxaparin_sodium.pdf ➜ 6 useful sections
[196/2798] Arovi_8_000_IU_80_mg_0_8_mL_solution_for_injection_in_pre-filled_syringe.pdf ➜ 6 useful sections
[197/2798] Arquist_125_microgram_per_actuation_pressurised_inhalation_suspension.pdf ➜ 6 useful sections
[198/2798] Artelac_3_2_mg_ml_Eye_Drops_Solution_Hypromellose.pdf ➜ 6 useful sections
[199/2798] Artelac_Preservative_Free_Single_Dose_Unit_Hypromellose.pdf ➜ 6 useful sections
[200/2798] Arthrotec_50_modified-release_tablets.pdf ➜ 6 useful sections
[201/2798] Arthrotec_75_modified-release_tablets.pdf ➜ 6 useful sections
[202/2798] Arthrotec_75_modified-release_tablets_diclofenac_sodium_misoprostol.pdf ➜ 6 useful sections
[203/2798] Artichoke-Milk_Thistle.pdf ➜ 6 useful sections
[204/2798] Asacolon_1000_mg_Suppositories_mesalazine.pdf ➜ 6 useful se

[289/2798] BELKYRA_10_mg_ml_solution_for_injection.pdf ➜ 6 useful sections
[290/2798] Belnifrem_1_5_mg_film-coated_tablets_cytisinicline.pdf ➜ 6 useful sections
[291/2798] Bendamustine_25_mg_and_100_mg_Powder_for_Concentrate_for_Solution_for_Infusion.pdf ➜ 6 useful sections
[292/2798] Bendamustine_25_mg_ml_concentrate_for_solution_for_infusion_bendamustine_hydrochloride.pdf ➜ 6 useful sections
[293/2798] Bendamustine_25_mg_Powder_for_concentrate_for_Solution_for_Infusion.pdf ➜ 6 useful sections
[294/2798] Bendamustine_25_mg_Powder_for_concentrate_for_Solution_for_Infusion_Bendamustine_100_mg_Powder_for_concentrate_for_Solution_for_Infusion.pdf ➜ 6 useful sections
[295/2798] Benylin_Herbal_Chesty_Coughs_Syrup.pdf ➜ 6 useful sections
[296/2798] Benzylpenicillin_sodium_600mg.pdf ➜ 6 useful sections
[297/2798] Benzylpenicillin_sodium_600_mg_Powder_for_Solution_for_Injection_Infusion_Benzylpenicillin_sodium_3000_mg_Powder_for_Solution_for_Injection_Infusion.pdf ➜ 6 useful sections
[298/2798

[380/2798] Budenofalk_9mg_gastro-resistant_granules_budesonide.pdf ➜ 6 useful sections
[381/2798] Budesonide_Teva_Pharma_0_5_mg_2_ml_Nebuliser_Suspension.pdf ➜ 6 useful sections
[382/2798] Budesonide_Teva_Pharma_0_5_mg_2_ml_Nebuliser_Suspension_Budesonide_Teva_Pharma_1_mg_2_ml_Nebuliser_Suspension.pdf ➜ 6 useful sections
[383/2798] Bufomix_Easyhaler_160_micrograms_4_5_micrograms_inhalation_Inhalation_Powder.pdf ➜ 6 useful sections
[384/2798] Bufomix_Easyhaler_320_micrograms_9_micrograms_inhalation_Inhalation_Powder.pdf ➜ 6 useful sections
[385/2798] Bufomix_Easyhaler_80_micrograms_4_5_micrograms_inhalation_Inhalation_Powder.pdf ➜ 6 useful sections
[386/2798] Bulexin_2_mg_0_5_mg_sublingual_tablets.pdf ➜ 6 useful sections
[387/2798] Bulexin_4_mg_1_mg_sublingual_tablets.pdf ➜ 6 useful sections
[388/2798] Bupivacaine_Heavy_5mg.pdf ➜ 6 useful sections
[389/2798] Bupivacaine_Hydrochloride_0_25_w_v_Solution_for_Injection_and.pdf ➜ 6 useful sections
[390/2798] Buplex_200_mg_film-coated_tablets

...

[2394/2798] Sodium_chloride_5mmol_ml_Oral_Solution.pdf ➜ 6 useful sections
[2395/2798] Sodium_Chloride_Intravenous_Infusion_BP_0_9_w_v_Active_substance_sodium_chloride.pdf ➜ 6 useful sections
[2396/2798] Sodium_Chloride_Intravenous_Infusion_BP_0_9_w_v_Solution_for_Infusion_Ecobag.pdf ➜ 6 useful sections
[2397/2798] Sodium_oxybate_500_mg_ml_oral_solution.pdf ➜ 6 useful sections
[2398/2798] Solaraze_3_gel.pdf ➜ 6 useful sections
[2399/2798] Solaraze_3_gel_diclofenac_sodium.pdf ➜ 6 useful sections
[2400/2798] Solaraze_3_gel_Diclofenac_Sodium_3_w_w.pdf ➜ 6 useful sections
[2401/2798] Solferol_800_IU_Soft_Capsules.pdf ➜ 6 useful sections
[2402/2798] Solferol_800_IU_Soft_Capsules_Solferol_1_000_IU_Soft_Capsules.pdf ➜ 6 useful sections
[2403/2798] Solifenacin_Clonmel_5mg_Film-coated_Tablets.pdf ➜ 6 useful sections
[2404/2798] Solifenacin_succinate_5_mg_film-coated_tablets.pdf ➜ 6 useful sections
[2405/2798] Solifenacin_succinate_5_mg_film-coated_tablets_Solifenacin_succinate_10_mg_film-coated

[2489/2798] Targin_15_mg_7_5_mg_prolonged-release_tablets.pdf ➜ 6 useful sections
[2490/2798] Targin_5_mg_2_5_mg_prolonged-release_tablets.pdf ➜ 6 useful sections
[2491/2798] Targin_5_mg_2_5_mg_prolonged-release_tablets_Targin_10_mg_5_mg_prolonged-release_tablets.pdf ➜ 6 useful sections
[2492/2798] Targin_60_mg_30_mg_prolonged-release_tablets.pdf ➜ 6 useful sections
[2493/2798] Targocid_200_mg_powder_and_solvent_for_solution_for_injection_infusion_or_oral_solution.pdf ➜ 6 useful sections
[2494/2798] TdaPBooster.pdf ➜ 6 useful sections
[2495/2798] Technescan_DMSA.pdf ➜ 6 useful sections
[2496/2798] Technescan_DTPA_20_8_mg_kit_for_radiopharmaceutical_preparation_.pdf ➜ 6 useful sections
[2497/2798] Technescan_HDP_Kit_for_radiopharmaceutical_preparation_.pdf ➜ 6 useful sections
[2498/2798] TechneScan_MAG3_1mg_kit_for_radiopharmaceutical_preparation.pdf ➜ 6 useful sections
[2499/2798] Technescan_MIBI_1_mg_kit_for_radiopharmaceutical_preparation.pdf ➜ 6 useful sections
[2500/2798] Teicoplan

[2585/2798] Trileptal_60_mg_ml_oral_suspension.pdf ➜ 6 useful sections
[2586/2798] Trimega_1000mg_Capsules.pdf ➜ 6 useful sections
[2587/2798] Trimethoprim_10mg_ml_Oral_Suspension.pdf ➜ 6 useful sections
[2588/2798] triptorelin.pdf ➜ 6 useful sections
[2589/2798] Triregol_coated_tablets.pdf ➜ 6 useful sections
[2590/2798] Tritace_1_25mg_Tablets.pdf ➜ 6 useful sections
[2591/2798] Tritace_1_25mg_Tablets_Tritace_2_5mg_Tablets.pdf ➜ 6 useful sections
[2592/2798] Trosyl_283_mg_ml_Nail_Solution.pdf ➜ 6 useful sections
[2593/2798] Trosyl_283_mg_ml_Nail_Solution_tioconazole.pdf ➜ 6 useful sections
[2594/2798] Trosyl_283_mg_ml_Nail_Solution_Tioconazole_283_mg_ml.pdf ➜ 6 useful sections
[2595/2798] Trusitev_SR_2_mg_prolonged-release_capsules_hard.pdf ➜ 6 useful sections
[2596/2798] Trusitev_SR_2_mg_prolonged-release_capsules_hard_Trusitev_SR_4_mg_prolonged-release_capsules_hard.pdf ➜ 6 useful sections
[2597/2798] TRUSOPT_20_mg_ml_eye_drops_solution.pdf ➜ 6 useful sections
[2598/2798] TRUSOPT_Pr

[2684/2798] Vesomni_6_mg_0_4_mg_modified_release_tablets.pdf ➜ 6 useful sections
[2685/2798] Vesomni_6_mg_0_4_mg_modified_release_tablets_solifenacin_succinate_tamsulosin_hydrochloride.pdf ➜ 6 useful sections
[2686/2798] Viacoram_3_5_mg_2_5mg_tablets.pdf ➜ 6 useful sections
[2687/2798] Viacoram_3_5_mg_2_5mg_tablets_Viacoram_7_mg_5_mg_tablets.pdf ➜ 6 useful sections
[2688/2798] Viaredin_25_mg_chewable_tablets.pdf ➜ 6 useful sections
[2689/2798] Viaredin_50_mg_film-coated_tablets.pdf ➜ 6 useful sections
[2690/2798] Vicks_Inhaler_Nasal_Stick.pdf ➜ 6 useful sections
[2691/2798] Vidisic_Preservative_Free_Single_Dose_Unit.pdf ➜ 6 useful sections
[2692/2798] Vildagliptin_Krka_50_mg_tablets_vildagliptin.pdf ➜ 6 useful sections
[2693/2798] Vincristine_Sulfate_1_mg_ml_Solution_for_Injection_or_Infusion_vincristine_sulfate.pdf ➜ 6 useful sections
[2694/2798] Viralief_50mg_g_cream.pdf ➜ 6 useful sections
[2695/2798] Viscolex_Syrup_250_mg_5_ml_Oral_Solution.pdf ➜ 6 useful sections
[2696/2798] Vitam

[2784/2798] Zomig_Rapimelt_2_5_mg.pdf ➜ 6 useful sections
[2785/2798] Zomig_Rapimelt_2_5_mg_orodispersible_tablets.pdf ➜ 6 useful sections
[2786/2798] Zonisamide_Rowex_25_mg_hard_capsules.pdf ➜ 6 useful sections
[2787/2798] Zonisamide_Rowex_25_mg_hard_capsules_Zonisamide_Rowex_50_mg_hard_capsules.pdf ➜ 6 useful sections
[2788/2798] Zopiclone_Grindeks_3_75_mg_film-coated_tablets.pdf ➜ 6 useful sections
[2789/2798] Zopiclone_Grindeks_3_75_mg_film-coated_tablets_Zopiclone_Grindeks_5_mg_film-coated_tablets.pdf ➜ 6 useful sections
[2790/2798] Zopitan_3_75_mg_Film-Coated_Tablets.pdf ➜ 6 useful sections
[2791/2798] Zopitan_3_75_mg_Film-Coated_Tablets_Zopitan_7_5_mg_Film-Coated_Tablets.pdf ➜ 6 useful sections
[2792/2798] Zovirax_200_mg_5_mL_Oral_Suspension.pdf ➜ 6 useful sections
[2793/2798] Zovirax_200_mg_5_mL_Oral_Suspension_aciclovir.pdf ➜ 6 useful sections
[2794/2798] Zovirax_200_mg_Dispersible_Tablets.pdf ➜ 6 useful sections
[2795/2798] Zovirax_800_mg_Dispersible_Tablets.pdf ➜ 6 useful se

## Embedding the Leaflet Sections

In this step, each extracted leaflet section is converted into a numerical vector using the `all-MiniLM-L12-v2` sentence transformer model. These embeddings will be used to enable semantic search and retrieval within the chatbot.

In [57]:
print("Preparing embedding generation...")

# Select the embedding model
model_name = "sentence-transformers/all-MiniLM-L12-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

print(f"Using embedding model: {model_name}")
print(f"Total sections to process: {len(all_docs)}")

# Start timer
start = time.time()

# Generate embeddings and build FAISS vector store
print("Generating embeddings and building FAISS vector store...")
db = FAISS.from_documents(all_docs, embedding_model)

# Convert the FAISS vector store into a retriever object,which can be used to find the top 4 most relevant document chunks based on a query
retriever = db.as_retriever(search_kwargs={"k": 4})

# End timer
end = time.time()
print(f"Embeddings successfully created in {end - start:.2f} seconds.")

Preparing embedding generation...
Using embedding model: sentence-transformers/all-MiniLM-L12-v2
Total sections to process: 16788
Generating embeddings and building FAISS vector store...
Embeddings successfully created in 902.29 seconds.


In [37]:
# Save the FAISS index and its metadata to local storage
db.save_local("faiss_index")

Reload the Vector Store

In [ ]:
# Reuse the same embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

# Load the previously saved FAISS index
db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever(search_kwargs={"k": 4})


## Creating the base function for the Drug Chatbot Using LLaMA3

### Steps

1. **Retrieve 4 relevant documents** from the FAISS vector store using the input query.

2. **Extract metadata** such as section titles and drug names from the matched documents.

3. **Dosage detection:**
   - If the query includes a specific dosage (e.g. "500 mg", "2 ml"), the function tries to locate documents (meds) that match that dosage in their filenames.
   - If found, only that document is used to answer; otherwise, the function returns a message: I couldn’t find this specific med with this dosage.

4. **Disambiguation (in case of repeated drug names; that is, multiple versions of the same med with different dosages or forms (pills, oral solution, etc):**

   - If there are multiple versions of the same drug (different dosages or formats), but the user **does not mention** a specific dosage or form, the function:
     - Uses just the first document if no ambiguity is detected, that is, there is just one version of the med the user is asking for.
     - If ambiguity **is** detected and the query ask for a **generic** answer, it uses all matching documents to answer.
     - Otherwise, it asks the user to clarify which version they mean.

5. **Builds a prompt** using the extracted context and sends it to the LLM via the Groq API (LLaMA3 model).

6. **Returns** the generated answer, along with metadata about the sources used.

In [192]:
API_KEY = "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"

def chat_with_llama3(query, max_retries=10, initial_delay=7):
    
    # Retrieve relevant documents from the vector store
    results = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in results])

    # Extract metadata (section titles and drug names)
    sources_info = []
    drug_names = []
    for doc in results:
        source = doc.metadata['source']
        lines = doc.page_content.splitlines()
        section_title = lines[1].strip() if len(lines) >= 2 else "Unknown Section"
        sources_info.append((source, section_title))

        base_name = os.path.basename(source).split("_")[0].lower()
        drug_names.append(base_name)

    counts = Counter(drug_names)
    repeated_drugs = [drug for drug, count in counts.items() if count > 1 and drug in query.lower()]

    # Check for specific dosage mentions
    dosage_match = re.search(r"(\d+)\s*(mg|g|mcg|μg|ml|microgram|micrograms)", query.lower())

    if dosage_match:
        number = dosage_match.group(1)

        # Create multiple patterns to match dosage in filenames
        patterns = [
            rf"[_]{number}[\s_]*mg",
            rf"[_]{number}[\s_]*ml",
            rf"[_]{number}[_]",
            rf"[_]{number}[\s_]*microgram",
            rf"[_]{number}[\s_]*micrograms"
        ]

        # Try to find a matching document
        matched_doc = next(
            (doc for doc in results if any(re.search(p, doc.metadata["source"].lower()) for p in patterns)),
            None
        )

        if matched_doc:
            results = [matched_doc]
            sources_info = [(matched_doc.metadata["source"], matched_doc.page_content.splitlines()[1].strip())]
        else:
            return (
                f"Sorry, I couldn’t find this specific med with {number} dosage. "
                "If you want to ask me for another dosage or just want to know general information, I will help you"
            ), sources_info

    else:
        # No dosage mentioned — handle ambiguity and specificity
        specific_words = [
            "mg", "g", "mcg", "μg", "ml", "microgram", "micrograms",
            "tablet", "tablets", "capsule", "capsules",
            "solution", "drops", "injection", "injectable",
            "cream", "ointment", "spray"
        ]

        query_mentions_specific = any(re.search(rf"\b{x}\b", query.lower()) for x in specific_words)

        if query_mentions_specific:
            # User specified form/dosage — use only the first result
            results = [results[0]]
            sources_info = [sources_info[0]]

        elif not repeated_drugs:
            # No conflict — use first document
            results = [results[0]]
            sources_info = [sources_info[0]]

        elif repeated_drugs and not query_mentions_specific:
            # If query is generic, use all documents
            generic_keywords = ["generic", "general", "generally", "overview", "summary"]
            if any(word in query.lower() for word in generic_keywords):
                pass  # Use all documents
            else:
                # Ambiguity without clarification — ask the user
                repeated = repeated_drugs[0]
                options = sorted(set(
                    os.path.basename(res.metadata['source']).replace(".pdf", "")
                    for res, name in zip(results, drug_names)
                    if name == repeated
                ))
                clarification_message = (
                    f"There are multiple versions of {repeated.capitalize()} available "
                    f"(e.g., {', '.join(options)}). Please specify which one (including dosage in mg) you mean or if you want to know general information about {repeated.capitalize()}."
                )
                return clarification_message, sources_info

    # Rebuild context after filtering
    context = "\n\n".join([doc.page_content for doc in results])

    # Build prompt for LLM
    prompt = f"""<s>[INST] You are a helpful medical assistant. 
A person is going to ask you something about a drug or medicine that you will have to answer in a conversational style 
Only use the context below to answer the question. Please answer briefly in no more than 6 lines. If you do not know the answer, say:
"I am sorry but right now I am afraid don't have that information. You are welcome to ask me another question". Do not invent the answer
For example if you do not know anything about a drug or medicine do not invent it and also if I ask you about things not related with medicines do not give that information cause you dont know it.

Context:
{context}

Question: {query}
Answer: [/INST]"""

    attempt = 0
    delay = initial_delay

    while attempt < max_retries:
        try:
            response = requests.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {API_KEY}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "llama3-8b-8192",
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.3,
                    "max_tokens": 300
                },
                timeout=60
            )
            
            data = response.json()
            if "choices" in data:
                answer = data["choices"][0]["message"]["content"]
                return answer.strip(), sources_info
        except Exception:
            pass

        time.sleep(delay)
        attempt += 1

    return "Error: No valid answer returned after retries.", sources_info


## Multilingual Functionality for the Drug Chatbot

This section extends the chatbot with multilingual support by detecting the user's language, translating the query to English (if necessary), and then translating the response back.

### Steps

1. **Language Detection**:  
   The function `detectar_idioma_con_confianza()` detects the user's language using both general models and Unicode-based character filters for Chinese, Japanese, and Korean.

2. **Translation Pipeline**:  
   If the input is not in English, it is translated using the M2M100 model from Facebook AI.  
  
3.  If the translated input is almost identical to the original, the system assumes it was already in English and skips translation.

4. **Chatbot Integration**:  
   The English version of the question is passed to `chat_with_llama3()`. The response is translated back into the user’s original language.


In [193]:
MODELO_TRADUCCION = "m2m100" 

In [194]:
# MODEL SETUP

model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# LANGUAGE DETECTION FUNCTION

def detectar_idioma_con_confianza(texto, umbral=0.65):
    
    # First: use general detection
    idiomas_generales = detect_langs(texto)
    print("General detection:", idiomas_generales)  
    
    if idiomas_generales and idiomas_generales[0].prob >= umbral:
        idioma_general = idiomas_generales[0].lang
        if idioma_general not in ["zh", "ja", "ko"]:
            return idioma_general  # Return the language
        
    # Manually check for specific character ranges (CJK)

    texto_chino = ''.join([c for c in texto if '\u4e00' <= c <= '\u9fff'])

    texto_japones = ''.join([c for c in texto if '\u3040' <= c <= '\u309F' or '\u30A0' <= c <= '\u30FF'])

    texto_coreano = ''.join([c for c in texto if '\uAC00' <= c <= '\uD7AF'])

    # If Japanese is deteted
    if texto_japones:
        idiomas = detect_langs(texto_japones)
        print("Japanese detected:", idiomas) 
        if idiomas and idiomas[0].prob >= umbral:
            return "ja"  

    # If Korean is detected
    if texto_coreano:
        idiomas = detect_langs(texto_coreano)
        print("Korean detected:", idiomas)
        if idiomas and idiomas[0].prob >= umbral:
            return "ko" 
        
    # If Chinese is detected 
    if texto_chino:
        idiomas = detect_langs(texto_chino)
        print("Chino detectado:", idiomas)  
        if idiomas and idiomas[0].prob >= umbral:
            return "zh"  
        
    # If Chinese, Japanese, or Korean are not detected, we return the general language
    return "en"  # Default fallback language ( English ) 


# TRANSLATION FUNCTION

def traducir_con_formato(texto, idioma_origen, idioma_destino):
    
    # 1. Detect and split text into list items and normal text
    partes = re.split(r'(\*\s+.*)', texto)  # This separates into normal text and list items

    partes_traducidas = []
    for parte in partes:
        parte = parte.strip()
        if parte.startswith("*"):
            
            # Translate list items, preserve '*'
            contenido = parte[1:].strip()  # Remove the asterisk
            contenido_traducido = traducir(contenido, idioma_origen, idioma_destino)
            partes_traducidas.append(f"* {contenido_traducido}")
            
        elif parte:  
            # Translate normal paragraphs
            contenido_traducido = traducir(parte, idioma_origen, idioma_destino)
            partes_traducidas.append(contenido_traducido)

    # 2. Rejoin translated parts
    texto_final = "\n".join(partes_traducidas)
    return texto_final

def son_similares(texto1, texto2, umbral=0.95):
    
    """Returns True if both texts are nearly identical"""
    return SequenceMatcher(None, texto1.lower(), texto2.lower()).ratio() >= umbral


def traducir(texto, idioma_origen, idioma_destino):
    with torch.no_grad():
        tokenizer.src_lang = idioma_origen
        inputs = tokenizer(texto, return_tensors="pt", truncation=True).to(device)
        translated = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.get_lang_id(idioma_destino),
            max_length=512
        )
        return tokenizer.decode(translated[0], skip_special_tokens=True)

# MULTILINGUAL CHATBOT WRAPPER

def chatbot_multilingue(pregunta_usuario, chatbot_funcion_ingles):
    
    idioma_usuario = detectar_idioma_con_confianza(pregunta_usuario)
    print("Detected language:", idioma_usuario)
    print (pregunta_usuario)
    if idioma_usuario == "en":
        
        # If input is English, proceed directly
        respuesta, pdfs = chat_with_llama3(pregunta_usuario)
        print(respuesta)
        return respuesta, pdfs
    else:
        print("Detected a non-English language.")

        # Translate the user question to English
        if MODELO_TRADUCCION == "m2m100":
            lang_origen = idioma_usuario
            lang_destino = "en"
        else:
            lang_origen = idioma_usuario
            lang_destino = "en"

        pregunta_en_ingles = traducir(pregunta_usuario, lang_origen, lang_destino)
        print("Translated question to English:", pregunta_en_ingles)
        
        # If the translated question is too similar to the original, assume it was English all along
        if son_similares(pregunta_usuario, pregunta_en_ingles):
            print("The question was already in English. Adjusting detection...")
            respuesta, pdfs = chat_with_llama3(pregunta_usuario)
            return respuesta, pdfs

        print(pregunta_en_ingles)
        
        # 2. Get response from chatbot in English
        respuesta_en_ingles, pdfs = chat_with_llama3(pregunta_en_ingles)
        print("Response in English:", respuesta_en_ingles)

        # 3. Translate response back to the user's language
        respuesta_traducida = traducir_con_formato(respuesta_en_ingles, lang_destino, lang_origen)

        # 4. Introductory message translated as well
        nombre_idioma = pycountry.languages.get(alpha_2=idioma_usuario).name
        frase_intro = f"Oh! It seems you want to speak in {nombre_idioma}. I'll answer in this language too."
        frase_intro_traducida = traducir(frase_intro, "en", idioma_usuario)

        # inal response with intro + answer
        respuesta_final = f"{frase_intro_traducida}\n\n{respuesta_traducida}"
        return respuesta_final, pdfs

## Example: Testing the Multilingual Drug Chatbot


In [195]:
# Replace this question with the one you wish to submit to the chatbot
question = "Can Aciclovir 500 mg cause neurological side effects?"

# Start timer
start = time.time()

# Call the multilingual chatbot
response, pdfs = chatbot_multilingue(question, chat_with_llama3)

# Stop timer
end = time.time()

# Display the chatbot's answer and source info
print("Chatbot response:\n")
print(response)

print("\n Information retrieved from the following PDFs:")
print(pdfs)

print(f"\n Total response time: {end - start:.2f} seconds")


General detection: [ca:0.9999944892380251]
Detected language: ca
Can Aciclovir 500 mg cause neurological side effects?
Detected a non-English language.
Translated question to English: Can Acyclovir 500 mg cause neurological side effects?
The question was already in English. Adjusting detection...
Chatbot response:

Yes, Aciclovir 500 mg can cause neurological side effects, such as headache, feeling giddy, balance problems, ataxia, dysarthria, confusion, agitation, tremors, myoclonus, convulsions, hallucinations, psychosis, drowsiness, and coma.

 Information retrieved from the following PDFs:
[('Aciclovir_500_mg_powder_for_solution_for_infusion.pdf', '4. Possible side effects')]

 Total response time: 6.60 seconds


In [196]:
# Replace this question with the one you wish to submit to the chatbot
question = "¿Qué síntomas deberían hacerme dejar de tomar Arcoxia inmediatamente?"

# Start timer
start = time.time()

# Call the multilingual chatbot
response, pdfs = chatbot_multilingue(question, chat_with_llama3)

# Stop timer
end = time.time()

# Display the chatbot's answer and source info
print("Chatbot response:\n")
print(response)

print("\n Information retrieved from the following PDFs:")
print(pdfs)

print(f"\n Total response time: {end - start:.2f} seconds")

General detection: [es:0.9999941423193642]
Detected language: es
¿Qué síntomas deberían hacerme dejar de tomar Arcoxia inmediatamente?
Detected a non-English language.
Translated question to English: What symptoms should make me stop taking Arcoxia immediately?
What symptoms should make me stop taking Arcoxia immediately?
Response in English: There are multiple versions of Arcoxia available (e.g., ARCOXIA_30_mg_film-coated_tablets, ARCOXIA_60_mg_film-coated_tablets, Arcoxia_30_mg_film-coated_tablets_Arcoxia_60_mg_film-coated_tablets, Arcoxia_30mg). Please specify which one (including dosage in mg) you mean or if you want to know general information about Arcoxia.
Chatbot response:

Oh! parece que quieres hablar en español. yo responderé en este idioma también.

Hay varias versiones de Arcoxia disponibles (por ejemplo, ARCOXIA_30_mg_film-coated_tablets, ARCOXIA_60_mg_film-coated_tablets, Arcoxia_30_mg_film-coated_tablets_Arcoxia_60_mg_film-coated_tablets, Arcoxia_30mg). Por favor, espec

In [198]:
# Replace this question with the one you wish to submit to the chatbot
question = "¿Qué síntomas deberían hacerme dejar de tomar Arcoxia 60 mg inmediatamente?"

# Start timer
start = time.time()

# Call the multilingual chatbot
response, pdfs = chatbot_multilingue(question, chat_with_llama3)

# Stop timer
end = time.time()

# Display the chatbot's answer and source info
print("Chatbot response:\n")
print(response)

print("\n Information retrieved from the following PDFs:")
print(pdfs)

print(f"\n Total response time: {end - start:.2f} seconds")

General detection: [es:0.9999945732992132]
Detected language: es
¿Qué síntomas deberían hacerme dejar de tomar Arcoxia 60 mg inmediatamente?
Detected a non-English language.
Translated question to English: What symptoms should make me stop taking Arcoxia 60 mg immediately?
What symptoms should make me stop taking Arcoxia 60 mg immediately?
Response in English: You should stop taking ARCOXIA 60 mg immediately if you develop any of the following signs: shortness of breath, chest pains, or ankle swelling that appear or get worse, yellowing of the skin and eyes (jaundice), severe or continual stomach pain or your stools become black, or an allergic reaction, which can include skin problems such as ulcers or blistering, or swelling of the face, lips, tongue, or throat which may cause difficulty in breathing.
Chatbot response:

Oh! parece que quieres hablar en español. yo responderé en este idioma también.

Debe dejar de tomar ARCOXIA 60 mg de inmediato si se desarrolla cualquiera de los sig

## Evaluation of the Multilingual Drug Chatbot 

This section evaluates the chatbot's performance by comparing its answers to a reference dataset (`Evaluation_questions.xlsx`).  
Each question is submitted to the chatbot, and the output is compared against the expected answer using:

- **Recall@1 (Document)**: whether the correct PDF was retrieved.
- **Recall@1 (Section)**: whether the correct section was retrieved.
- **BERTScore**: semantic similarity between the chatbot's answer and the reference answer.

We evaluate the chatbot’s performance using different temperature settings and various LLM models.
Final results are stored in an output file and mean metrics are computed for overall performance analysis.

In [199]:
# Load the evaluation dataset from Excel
df = pd.read_excel("Evaluation_questions.xlsx")

# Preview the first 10 entries
df.head(10)

# Define evaluation functions
def recall_doc_match(predicted_sources, correct_pdf):
    if not predicted_sources:
        return 0
    pred_pdf, _ = predicted_sources[0]
    return int(pred_pdf.strip().lower() == correct_pdf.strip().lower())

def recall_section_match(predicted_sources, correct_section):
    if not predicted_sources:
        return 0
    _, pred_section = predicted_sources[0]
    return int(pred_section.strip().lower() == correct_section.strip().lower())


In [202]:
# Store results
all_results = []

# Iterate through all evaluation samples
for i, row in df.iterrows():
    question = row['Question']
    ref_answer = row['Ground truth answer']
    ref_pdf = row['Document source']
    ref_section = row['Section']

    print(f"\n[{i+1}/{len(df)}] Evaluating: {question}")
    time.sleep(2)

    try:
        # Get response from the chatbot
        response, pdfs = chatbot_multilingue(question, chat_with_llama3)

        # Evaluate document and section recall
        r_doc = recall_doc_match(pdfs, ref_pdf)
        r_sec = recall_section_match(pdfs, ref_section)

        # Calculate BERTScore
        P, R, F1 = bert_score([response], [ref_answer], lang="multilingual", verbose=False)
        bert = float(F1[0])

    except Exception as e:
        print(f"Error on question {i}: {e}")
        r_doc, r_sec, bert = 0, 0, 0.0
        response, pdfs = "ERROR", []

    # Store result
    all_results.append({
        "Question": question,
        "Ground Truth Answer": ref_answer,
        "Chatbot Answer": response,
        "Correct PDF": ref_pdf,
        "Correct Section": ref_section,
        "Returned PDF": pdfs[0][0] if pdfs else "",
        "Returned Section": pdfs[0][1] if pdfs else "",
        "Recall@1 (Document)": r_doc,
        "Recall@1 (Section)": r_sec,
        "BERTScore": round(bert, 4)
    })

# Convert results to DataFrame
results_df = pd.DataFrame(all_results)

results_df.to_excel("Evaluation_Results.xlsx", index=False)

# Compute mean metrics
recall_doc_avg = results_df["Recall@1 (Document)"].mean()
recall_sec_avg = results_df["Recall@1 (Section)"].mean()
bert_avg = results_df["BERTScore"].mean()

# Print global metrics
print("\nGLOBAL METRICS:")
print(f"Recall@1 (Document): {recall_doc_avg:.2f}")
print(f"Recall@1 (Section): {recall_sec_avg:.2f}")
print(f"Average BERTScore: {bert_avg:.4f}")

# Save summary metrics
summary_metrics_df = pd.DataFrame({
    "Metric": ["Recall@1 (Document)", "Recall@1 (Section)", "BERTScore Average"],
    "Value": [recall_doc_avg, recall_sec_avg, bert_avg]
})

summary_metrics_df.to_excel("Evaluation_Summary.xlsx", index=False)




[1/33] Evaluating: What should a patient do if they experience sudden wheezing or swelling of the face after taking Acerycal 5 mg ?
General detection: [en:0.9999967211543943]
Detected language: en
What should a patient do if they experience sudden wheezing or swelling of the face after taking Acerycal 5 mg ?
If a patient experiences sudden wheezing or swelling of the face after taking Acerycal 5mg, they should stop taking the medicinal product at once and tell their doctor immediately.

[2/33] Evaluating: How should ACULAR eye drops be applied properly?
General detection: [en:0.9999965193150304]
Detected language: en
How should ACULAR eye drops be applied properly?
To apply ACULAR eye drops properly, first wash your hands and tilt your head back. Gently pull your lower eyelid down to create a small pocket, then turn the bottle upside down and squeeze one drop into each affected eye. Let go of your lower lid and close your eye for 30 seconds. If a drop misses your eye, try again.

[3/3


[18/33] Evaluating: Are there any age restrictions for using Echinaforce Cold & Flu drops?
General detection: [en:0.9999962311917732]
Detected language: en
Are there any age restrictions for using Echinaforce Cold & Flu drops?
Yes, there is an age restriction for using Echinaforce Cold & Flu drops. Do not take Echinaforce if you are under 12 years of age.

[19/33] Evaluating: Can Epanutin Infatabs 50 mg affect how other medicines work?
General detection: [en:0.999993893169271]
Detected language: en
Can Epanutin Infatabs 50 mg affect how other medicines work?
Yes, Epanutin Infatabs 50 mg can affect how other medicines work. It can interact with many medications, including antacids, anticoagulants, antiplatelets, antiepileptics, antibiotics, antidepressants, antihistamines, and many others. Your doctor may need to monitor the amount of phenytoin in your blood to help decide if any of these drugs are affecting your treatment.

[20/33] Evaluating: Are there any dietary restrictions I shou

General detection: [en:0.8571396949001078, af:0.1428595110920503]
Detected language: en
Is it safe to take Valoid 50 mg if I’ve had too much to drink?
No, it's not safe to take Valoid 50 mg if you've had too much to drink. The anti-vomiting properties of cyclizine may increase the toxicity of alcohol.

GLOBAL METRICS:
Recall@1 (Document): 0.91
Recall@1 (Section): 0.88
Average BERTScore: 0.8043
